In [1]:
from games_setup import *
from SBMLLint.common import constants as cn
from SBMLLint.common.molecule import Molecule, MoleculeStoichiometry
from SBMLLint.common.reaction import Reaction
from SBMLLint.games.som import SOM
from SBMLLint.common.simple_sbml import SimpleSBML

import collections
import itertools
import networkx as nx
import numpy as np
import pandas as pd
from sympy.matrices import Matrix, eye
#
from SBMLLint.common.stoichiometry_matrix import StoichiometryMatrix
from SBMLLint.games.mesgraph import MESGraph
from SBMLLint.games.message import Message, SOMStoichiometry, SOMReaction

from scipy.linalg import lu, inv

Current Directory: /Users/woosubshin/Desktop/ModelEngineering/SBMLLint/notebook


In [2]:
simple = load_file_from_curated_data(61)
for r in simple.reactions:
  if r.category != cn.REACTION_BOUNDARY:
    print(r.makeIdentifier(is_include_kinetics=False))

vinGlc: GlcX0 -> GlcX
vGlcTrans: GlcX -> 59.00 Glc
vHK: ATP + Glc -> G6P + ADP
vPGI: G6P -> F6P
vPFK: F6P + ATP -> FBP + ADP
vALD: FBP -> GAP + DHAP
vTIM: DHAP -> GAP
vGAPDH: GAP + NAD -> NADH + BPG
vlpPEP: BPG + ADP -> PEP + ATP
vPK: ADP + PEP -> Pyr + ATP
vPDC: Pyr -> ACA
vADH: NADH + ACA -> NAD + EtOH
vdifEtOH: 59.00 EtOH -> EtOHX
voutEtOH: EtOHX -> P
vlpGlyc: DHAP + NADH -> Glyc + NAD
vdifGlyc: 59.00 Glyc -> GlycX
voutGlyc: GlycX -> P
vdifACA: 59.00 ACA -> ACAX
voutACA: ACAX -> P
vlacto: CNX + ACAX -> P
vinCN: CNX0 -> CNX
vstorage: ATP + G6P -> ADP
vconsum: ATP -> ADP
vAK: ATP + AMP -> 2.00 ADP


In [3]:
m = Message(simple)

In [5]:
s = np.array([[-1, 0, 0, 1],
            [-1, -1, 1, 1]])
sd = pd.DataFrame(s, index=['R1', 'R2'], columns=['APT', 'Glc', 'G6P', 'ADP'])
sd

,APT,Glc,G6P,ADP
R1,-1,0,0,1
R2,-1,-1,1,1


In [6]:
s_ = np.array([[-1, 0, 0, 1],
            [0, -1, 1, 0]])
sd_ = pd.DataFrame(s_, index=['R1', 'R2*'], columns=['APT', 'Glc', 'G6P', 'ADP'])
sd_

,APT,Glc,G6P,ADP
R1,-1,0,0,1
R2*,0,-1,1,0


In [7]:
s__ = np.array([[-1, 0, 0, 1, 1],
            [-1, -1, 1, 1, 0]])
sd__ = pd.DataFrame(s__, index=['R1', 'R2'], columns=['APT', 'Glc', 'G6P', 'ADP', 'Pi'])
sd__

,APT,Glc,G6P,ADP,Pi
R1,-1,0,0,1,1
R2,-1,-1,1,1,0


In [8]:
s_new = np.array([[-1, 0, 0, 1, 1],
            [0, -1, 1, 0, -1]])
sd_new = pd.DataFrame(s_new, index=['R1', 'R2*'], columns=['APT', 'Glc', 'G6P', 'ADP', 'Pi'])
sd_new

,APT,Glc,G6P,ADP,Pi
R1,-1,0,0,1,1
R2*,0,-1,1,0,-1


In [20]:
simple2 = load_file_from_curated_data(383)
for r in simple2.reactions:
  if r.category != cn.REACTION_BOUNDARY:
    print(r.makeIdentifier(is_include_kinetics=False))
m2 = Message(simple2)

PGA_prod_Vc: RuBP + CO2 + 2.00 NADPH -> 2.00 PGA
PGA_prod_Vo: RuBP + CO2 + 2.00 NADPH -> 1.50 PGA
PGA_cons: PGA -> RuBP
NADPH_prod: NADP -> NADPH


In [22]:
e = np.array([[-1, -2, 1],
            [-1, -2, 0.5]])
ed = pd.DataFrame(e, index=['R1', 'R2'], columns=['CO2', 'NADPH', 'PGA'])
ed

,CO2,NADPH,PGA
R1,-1.0,-2.0,1.0
R2,-1.0,-2.0,0.5


In [49]:
ef = np.array([[-1, -2, 1],
            [0, 0, -0.5]])
ef_d = pd.DataFrame(ef, index=['R1*', 'R2*'], columns=['CO2', 'NADPH', 'PGA'])
ef_d

,CO2,NADPH,PGA
R1*,-1.0,-2.0,1.0
R2*,0.0,0.0,-0.5


In [33]:
p = np.identity(2)
pdd = pd.DataFrame(p, index=['R1', 'R2'], columns=['R1', 'R2'])
pdd

,R1,R2
R1,1.0,0.0
R2,0.0,1.0


In [35]:
l = np.array([[1.0, 0.0],
            [-1.0, 1.0]])

ld = pd.DataFrame(l, index=['R1', 'R2'], columns=['R1', 'R2'])
ld

,R1,R2
R1,1.0,0.0
R2,-1.0,1.0


In [46]:
rref = np.array([[1.0, 0.0, 0.0, 1.0, -1.0],
            [0.0, 1.0, 1.0, 1.0, -1.0],
            [0.0, 0.0, -1.0, -1.0, 1.0]])

rref_d = pd.DataFrame(rref, index=['R1', 'R2', 'R3'],
                      columns=['A', 'B', 'C', 'D', 'E'])
rref_d

,A,B,C,D,E
R1,1.0,0.0,0.0,1.0,-1.0
R2,0.0,1.0,1.0,1.0,-1.0
R3,0.0,0.0,-1.0,-1.0,1.0


In [39]:
np.array([0.05, 0.25, 0.7]) * 0.18

array([0.009, 0.045, 0.126])

In [43]:
0.168 + 0.045 + 0.0224 + 0.036

0.27140000000000003

In [48]:
rreff = np.array([[1.0, 0.0, 0.0, 1.0, -1.0],
            [0.0, 1.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, -1.0, -1.0, 1.0]])

rreff_d = pd.DataFrame(rreff, index=['R1', 'R2*', 'R3'],
                      columns=['A', 'B', 'C', 'D', 'E'])
rreff_d

,A,B,C,D,E
R1,1.0,0.0,0.0,1.0,-1.0
R2*,0.0,1.0,0.0,0.0,0.0
R3,0.0,0.0,-1.0,-1.0,1.0
